In [ ]:
!pip3 install biopython==1.77

     |████████████████████████████████| 2.3MB 5.1MB/s 


In [ ]:
from Bio import SeqIO
from Bio import Entrez
from Bio.Seq import Seq
from Bio.GenBank.Record import Record
from Bio.SeqRecord import SeqRecord
from Bio.Alphabet import IUPAC
from Bio.SeqFeature import SeqFeature, FeatureLocation
from Bio.Alphabet import generic_dna
from collections import OrderedDict
import pandas as pd
import urllib
import urllib.request
import numpy as np


Задача: аннотировать фрагмент генома бактерии и как можно точнее определить таксон, к которому бактерия принадлежит. 

**Этапы работы**

1) Получаем открытые рамки считывания (ОРС) через Artemis (Create->Mark open reading frames). Выделяем их все и смещаем к старт-кодонам (Edit->Trim selected features->To Any). Переименовываем их и задаем  Edit ->Automatically Create Gene Names (я задал my_orf с шагом 1)

2) загружаем ORF из Artemis в blastp с параметрами:
DataBase: uniprot/swiss-Prot
Organism: Bacteria
Max target sequences: 50
Expect threshold: 0.5
Word size: 6
Matrix: BLOSUM62

Ожидание составило минут 10. 
Далее выгружаем данные в формате HitTable.csv и начинаем анализировать



Ниже мы считываем данные из BLASP,а также результат работы artemis для координат CDS

In [ ]:
#read all data info
names=['query acc.ver',  'identity', 'alignment length','mismatches', 'gap opens', 'q_start', 'q_end','s. start', 's. end', 'evalue', 'bit score', '% positives']
swiss = pd.read_csv('HitTableUprt.csv', names=names, index_col=0,)
fasta_sequences = SeqIO.parse(open('DNA.fasta'),'fasta')
with open('DNA.fasta') as out_file:
    for fasta in fasta_sequences:
        name = fasta.id 
        sequence = str(fasta.seq)
coordinates = pd.DataFrame()
with open('amino_acids.fasta', 'r') as text:
  for line in text:
    if 'my_orf' in line:
      line = line.strip().split(' ')
      cds = line[0][1:]
      line = np.array(line[4].split(':')).astype(int)
      coordinates[cds] = line
     
coordinates.index = ['begin', 'end']
coordinates

,my_orf001,my_orf001c,my_orf003,my_orf004,my_orf005,my_orf006c,my_orf007,my_orf008,my_orf009,my_orf002c,my_orf011,my_orf012c,my_orf013,my_orf014c,my_orf015,my_orf016,my_orf017,my_orf018,my_orf019c,my_orf003c,my_orf021,my_orf004c,my_orf023,my_orf005c,my_orf025,my_orf026,my_orf027,my_orf029,my_orf030c,my_orf031c,my_orf032,my_orf033c,my_orf034,my_orf035c,my_orf036,my_orf037c,my_orf038c,my_orf039,my_orf040,my_orf041c,...,my_orf064,my_orf065c,my_orf066,my_orf067c,my_orf068,my_orf069c,my_orf009c,my_orf071,my_orf010c,my_orf073,my_orf074c,my_orf011c,my_orf076,my_orf077,my_orf078c,my_orf012,my_orf080,my_orf081,my_orf082,my_orf083,my_orf084,my_orf085,my_orf086c,my_orf087,my_orf088c,my_orf089,my_orf090,my_orf091,my_orf092c,my_orf093,my_orf094,my_orf095,my_orf096c,my_orf097,my_orf098,my_orf013c,my_orf100c,my_orf101c,my_orf102,my_orf103c
begin,21,368,1573,2373,3289,17112,26915,6348,7555,8033,8256,8426,8961,9398,10185,11006,11710,12718,12851,13249,13493,13843,14199,14366,15177,15891,16981,19179,19783,20480,20574,21294,22253,22825,23414,23629,24047,24747,26205,26695,...,41030,41622,41941,43069,43994,44265,44521,45038,45398,45618,46111,46874,47099,47458,48279,49417,49813,50487,50934,51595,52442,52801,54036,54454,55601,55997,56941,58066,58962,58981,59745,61253,62236,63136,64152,66053,67012,68384,68681,69465
end,1460,685,1911,3050,4437,17633,27238,6803,8232,8425,8807,8812,9794,9841,10991,11719,12699,13479,13156,13617,14197,14205,15161,14677,15824,16625,19152,19748,20295,21022,21116,22055,22744,23595,23653,23934,25522,25055,26654,27723,...,41920,41924,42885,43824,45019,44648,45057,45586,45736,46049,46917,47176,47566,48231,50222,49758,50151,50915,51572,51912,52804,54435,54416,55710,55897,56944,58050,58797,59606,59487,60911,63022,62550,63648,66788,66364,68406,68698,69898,69782


Ниже - фильтр для работы BLASTP. Для него мы задаем порог identity 35% и e-value <0.001

In [ ]:
def good_cds_selection(data_blast): 
  good = pd.DataFrame(columns = data_blast.columns)
  for cds in data_blast.index:
    curr_cds = pd.DataFrame(data_blast.loc[cds])
    if curr_cds.shape[1] == 1:
      curr_cds = curr_cds.T
    if ((curr_cds.loc[:,'identity'].iloc[0] > 35) & (curr_cds.loc[:,'evalue'].iloc[0] < 0.001)):
        good.loc[cds] = list(curr_cds.iloc[0])
  return good
good_cds = good_cds_selection(swiss)


In [ ]:
good_cds

,query acc.ver,identity,alignment length,mismatches,gap opens,q_start,q_end,s. start,s. end,evalue,bit score,% positives
my_orf005,P96608.1,53.439,378,176,0,5,382,4,381,1.990000e-147,424.0,70.63
my_orf007,P96683.1,44.643,224,124,0,5,228,3,226,3.200000e-64,201.0,68.30
my_orf009,B5F9Z0.1,38.462,208,121,4,5,206,5,211,2.070000e-43,148.0,58.65
my_orf011,P41027.1,52.795,161,76,0,19,179,17,177,1.440000e-57,180.0,72.05
my_orf013,O07541.1,40.000,120,70,1,156,275,117,234,1.850000e-17,83.2,55.83
my_orf015,P42968.4,52.653,245,116,0,22,266,5,249,2.630000e-90,270.0,71.84
my_orf016,P60495.1,47.111,225,114,3,6,227,10,232,9.450000e-74,226.0,69.78
my_orf017,Q7WY77.2,48.624,327,160,4,3,322,1,326,2.020000e-105,313.0,67.89
my_orf018,Q81BT0.1,89.723,253,26,0,1,253,1,253,8.910000e-173,478.0,94.07
my_orf025,B7JDC3.1,93.953,215,13,0,1,215,1,215,2.980000e-150,417.0,97.67


Для всех хороших находок удаляем пересекающиеся с ними CDS с приоритетом по identity. Также необходимо было учесть tRNA из tRNA-scan и/или Aragorn, но оба тула ничего не нашли (файлы прилагаю)


In [ ]:
def intersect(st1, end1, st2, end2):
      r = ((st1 <= end2) & (end1 >= end2))
      l = ((end1 >= st2) & (st1 <= st2))
      return (r | l)

print(intersect(5, 10, 7,8)) #must be true
print(intersect(5, 10, 2,8)) #must be true
print(intersect(5, 10, 8, 13)) #must be true
print(intersect(5, 10, 100, 200))#must be false
print('________________________________________')

def parse_intersections(findings, coord): 
  all_cds, good = list(findings.index), []
  findings = findings.sort_values(by='identity', ascending=False)
  coord = coord.T.loc[findings.index]
  while len(all_cds) != 2:
     curr_cds = all_cds[0]
     all_cds.remove(curr_cds)
     good.append(curr_cds)
     for cds in all_cds:
       if intersect(coord.loc[curr_cds, 'begin'], coord.loc[curr_cds, 'end'],
                    coord.loc[cds, 'begin'], coord.loc[cds, 'end']):
         all_cds.remove(cds)
  return good
unitersected = parse_intersections(good_cds, coordinates)
coordinates.T.loc[unitersected].shape

True
True
True
False
________________________________________


(34, 2)

Участки между CDS заносим в отдельный файл для BLASTN и запоминаем для них координаты

In [ ]:
agtc_fa, name_fa = [],  []
good = coordinates.T.loc[unitersected]
spaces_coord = pd.DataFrame()
for cds in range(len(unitersected)-1):
  agtc_fa.append(sequence[good.iloc[cds, 1]:good.iloc[cds+1, 0]])
  name_fa.append('seq'+str(cds))
  spaces_coord['seq'+str(cds)] = [coordinates.T.loc[unitersected].iloc[cds, 1], coordinates.T.loc[unitersected].iloc[cds+1, 0]]
ofile = open("seq_nucl_spaces.fasta", "w")
spaces_coord = spaces_coord.T
spaces_coord.columns = ['begin', 'end']
for i in range(len(agtc_fa)):#
      ofile.write(">" + name_fa[i] + "\n" +agtc_fa[i] + "\n")
ofile.close()

Результат для BLASTN пропускаем через фильтр по identity и e-value. Так как мы получаем выравнивание и не имеем информации о том, где на самом деле лежит начало важного учатска и его конец, то необходимо учесть координаты выравнивания (и следить за тем, чтобы ничего не пересеклось)

In [ ]:
names=[' ', 'query acc.ver',  'identity', 'alignment length','mismatches', 'gap opens', 'q_start', 'q_end','s. start', 's. end', 'evalue', 'bit score']
blastn = pd.read_csv('blastn.csv', index_col=0, names=names)
blastn = good_cds_selection(blastn)
blastn = blastn.drop('seq0') #много мисматчей и вообще вызывает подозрения
for x in blastn.index:
  blastn.loc[x, 'q_start'] += spaces_coord.loc[x, 'begin'] 
  blastn.loc[x, 'q_end'] += spaces_coord.loc[x, 'begin'] 

temp = blastn.loc[:,['q_start',	'q_end']]
temp.columns = ['begin', 'end']
all_coordinates = pd.concat([coordinates.T.loc[unitersected], temp]).sort_values(by='begin')
annotation_of_cds = pd.concat([blastn, good_cds.loc[unitersected]])


для orf с низким identity попробуем запустить interPro (ниже указаны только те, для которых было что-то найдено, для остальных None predicted)

In [ ]:
intrtPro_result = {'my_orf064': 'Transcription regulator HTH, AraC- type (IPR020449)',
                   'my_orf009': 'Copper homeostasis protein CutC (IPR005627)',
                   'my_orf029': 'Peptidase S26A, signal peptidase I (IPR000223)',
                   'my_orf011': 'Peptidase S26A, signal peptidase I (IPR000223)'}



Создаем аннотацию

In [ ]:
sequence_object = Seq(sequence, generic_dna)
record = SeqRecord(sequence_object, id=' ', name='Example',annotations={"molecule_type": "DNA", "organism": "Bacillus subtilis"},description='bacteria Bacillus subtilis')
for cds in all_coordinates.index:
  print(cds)
  if 'c' in cds:
        strand = -1
  else:
        strand = 1
  info = (str(annotation_of_cds.loc[cds,'identity']) + ', ' +'e-value: ' + str(annotation_of_cds.loc[cds,'evalue']) + ', ' + '% positives: ' + str(annotation_of_cds.loc[cds,'% positives']))
  if  cds[:3] == 'seq':
    handle = Entrez.efetch(db="nucleotide", id=annotation_of_cds.loc[cds, 'query acc.ver'], rettype="gb", retmode="text")
    for seq_record in SeqIO.parse(handle, "gb"):
      info = (seq_record.annotations['organism'] + " " + info)
    qualifier=OrderedDict([('gene',[annotation_of_cds.loc[cds, 'query acc.ver']]),('product',[seq_record.description]),('note',[info])])
    feature = SeqFeature(FeatureLocation(start=int(all_coordinates.loc[cds, 'begin']),end=int(all_coordinates.loc[cds, 'end']), strand=strand), type="CDS",qualifiers=qualifier)
    record.features.append(feature)
  elif cds in intrtPro_result.keys():
    with urllib.request.urlopen('http://www.uniprot.org/uniprot/'+ annotation_of_cds.loc[cds, 'query acc.ver'] + '.xml') as url:
        link_info = SeqIO.read(url, "uniprot-xml")
    qualifier=OrderedDict([('gene',[intrtPro_result[cds]]),('product',[link_info.description]),('note',[info+'interPRO result']),('translation', [link_info.seq])])
    feature = SeqFeature(FeatureLocation(start=int(all_coordinates.loc[cds, 'begin']),end=int(all_coordinates.loc[cds, 'end']), strand=strand), type="CDS",qualifiers=qualifier)
    record.features.append(feature)
    print('iPro')


  else: 
      with urllib.request.urlopen('http://www.uniprot.org/uniprot/'+ annotation_of_cds.loc[cds, 'query acc.ver'] + '.xml') as url:
        link_info = SeqIO.read(url, "uniprot-xml")
      if 'organism' in link_info.annotations:
        info += link_info.annotations['organism']
      if 'gene_name_primary' in link_info.annotations:
        qualifier=OrderedDict([('gene',[link_info.annotations['gene_name_primary']]),('product',[link_info.description]),('note',[info]),('translation', [link_info.seq])])
      else: 
        qualifier=OrderedDict([('gene',[link_info.description]),('product',[link_info.description]),('note',[info]),('translation', [link_info.seq])])

      feature = SeqFeature(FeatureLocation(start=int(all_coordinates.loc[cds, 'begin']),end=int(all_coordinates.loc[cds, 'end']), strand=strand), type="CDS",qualifiers=qualifier)
      record.features.append(feature)
 
with open('Nartsev_annot.gb', 'w') as output_file:
  SeqIO.write(record, output_file, 'genbank')

my_orf005
my_orf009
iPro
my_orf011
iPro
seq3


/usr/local/lib/python3.7/dist-packages/Bio/Entrez/__init__.py:668: UserWarning: 
Email address is not specified.

To make use of NCBI's E-utilities, NCBI requires you to specify your
email address with each request.  As an example, if your email address
is A.N.Other@example.com, you can specify it as follows:
   from Bio import Entrez
   Entrez.email = 'A.N.Other@example.com'
In case of excessive usage of the E-utilities, NCBI will attempt to contact
a user at the email address provided before blocking access to the
E-utilities.
  UserWarning,


my_orf013
seq4
my_orf015
my_orf016
seq6
my_orf018
seq7
my_orf025
seq8
my_orf026
seq9
my_orf027
my_orf029
iPro
seq11
my_orf030c
seq12
my_orf034
seq13
my_orf035c
seq14
my_orf038c
seq15
my_orf041c
my_orf007
seq16
my_orf050
seq17
my_orf051
seq18
my_orf053
seq19
my_orf054c
seq20
my_orf057c
seq21
my_orf063
seq22
my_orf064
iPro
seq23
my_orf074c
seq24
my_orf078c
seq25
my_orf081
my_orf082
my_orf083
seq28
my_orf085
seq29
my_orf089
seq30
my_orf094
seq31
my_orf095
seq32
my_orf098


Считываем информацию из RAST

In [ ]:
rast_cds = pd.DataFrame()
with open ('rast.txt') as text:
  for i, line in enumerate(text):
    line = line.split('	')[1][3:]
    coord_gene = np.array(line.split('_')).astype(int)
    if coord_gene[0] > coord_gene[1]:
      coord_gene[0], coord_gene[1] = coord_gene[1], coord_gene[0]
      rast_cds[i] = coord_gene[0], coord_gene[1]
    else:
      rast_cds[i] = coord_gene[0], coord_gene[1]

rast_cds = rast_cds.T
rast_cds.index = range(len(rast_cds.index))

In [ ]:
rast_cds

,0,1
0,21,1460
1,1573,1911
2,1979,2092
3,2391,3050
4,3289,4437
...,...,...
73,61268,63022
74,63136,63648
75,64152,66788
76,67012,68391


Вспомогательная функция для подсчета IoU (в числитель идет пересечение)

In [ ]:
def covering(st1, end1, st2, end2):
  if st1 > end1:
    st1, end1 = end1, st1
  if st2 > end2:
    st2, end2 = end2, st2
  if (st1 < st2) and (end2 > end1):
    overlap = end1 - st2
  elif (st2>st1) and (end1>end2):
    overlap = end2-st2
  elif (st2<st1) and (end1>end2):
    overlap = end2 - st1
  else:
    overlap = end1 - st1
  if overlap < 0:
    return 0
  else:
    return overlap+1 

covering(1,10,5,7) #for test
covering(2,10,1,100)#for test
covering(4,7,6,2)#for test
covering(4,7,6,2)#for test
covering(10,15,12,17)#for test
covering(10, 100, 200,300)#for test
covering(1, 10, 1, 10)#for test

10

Считаем IoU (поделить сумму длин пересекающихся участков на сумму объединения по всем генам)

In [ ]:
inter = []
denominator = []
s = 0
for i in rast_cds.index:
  maximum = 0
  for k in range(len(all_coordinates.index)): 
    value = covering(rast_cds.iloc[i,0], rast_cds.iloc[i,1],
                 all_coordinates.iloc[k,0], all_coordinates.iloc[k,1])
    if value > maximum:
      maximum = value
      s = k
  denominator.append( (rast_cds.iloc[i,1] - rast_cds.iloc[i,0]) + (all_coordinates.iloc[s,1] - all_coordinates.iloc[s,0]) - maximum)
  inter.append( maximum)


In [ ]:
iou = sum(inter)/sum(denominator)
iou

0.40335619237930875

Посчитаем, сколько совпало позиций начала и конца:

In [ ]:
st = 0
for i in rast_cds.index:
   for k in range(len(all_coordinates.index)):
     if rast_cds.iloc[i,0] == all_coordinates.iloc[k,0]:
       st+=1
end = 0
for i in rast_cds.index:
   for k in range(len(all_coordinates.index)):
     if rast_cds.iloc[i,1] == all_coordinates.iloc[k,1]:
       end+=1



st - сколько совпало начальных позиций
end - сколько совпало конечных позиций

In [ ]:
st, end

(25, 28)

Ниже аналогичный рассчет для Prokka



In [ ]:
prokka = {}
i=0
with open ('Galaxy108-_Prokka_on_data_100_tbl (1).txt') as text:
  for line in text:
    if 'CDS' in line:
      line = line.split('	')
      prokka[i] = [int(x) for x in line[:2]]
      i+=1
prokka = pd.DataFrame(prokka).T

In [ ]:
inter = []
denominator = []
s = 0
for i in prokka.index:
  maximum = 0
  for k in range(len(all_coordinates.index)): 
    value = covering(prokka.iloc[i,0], prokka.iloc[i,1],
                 all_coordinates.iloc[k,0], all_coordinates.iloc[k,1])
    if value > maximum:
      maximum = value
      s = k
  denominator.append( (prokka.iloc[i,1] - prokka.iloc[i,0]) + (all_coordinates.iloc[s,1] - all_coordinates.iloc[s,0]) - maximum)
  inter.append( maximum)


In [ ]:
iou = sum(inter)/sum(denominator)
iou

0.5176267853144197

посмотрим на количество совпадающих позиций

In [ ]:
st = 0
for i in prokka.index:
   for k in range(len(all_coordinates.index)):
     if prokka.iloc[i,0] == all_coordinates.iloc[k,0]:
       st+=1
end = 0
for i in prokka.index:
   for k in range(len(all_coordinates.index)):
     if prokka.iloc[i,1] == all_coordinates.iloc[k,1]:
       end+=1


In [ ]:
st, end

(17, 33)

Вывод: 
Организм - Bacillus subtilis. Количество хороших находок в результате работы blast - 30. Еще 4 находки были уточнены с помощью InterPro. 

Наибольшее совпадение по по проаннотированным участкам с аннотацией с результатом работы Prokka

In [ ]:
prokka_fuc = []
with open ('Galaxy108-_Prokka_on_data_100_tbl (1).txt') as text:
  for line in text:
    if (('product' in line) and ('hypothetical' not in line)):
      line = line.strip().split('	')
      prokka_fuc.append(line[1])
print(len(prokka_fuc))
prokka_fuc


33


['Putative acyl-CoA dehydrogenase YdbM',
 'Copper homeostasis protein CutC',
 'Signal peptidase IB',
 'HTH-type transcriptional regulator KipR',
 '5-oxoprolinase subunit B',
 '5-oxoprolinase subunit C',
 '5-oxoprolinase subunit A',
 'Pyrrolidone-carboxylate peptidase',
 'NAD-dependent protein deacetylase',
 'Penicillin-binding protein H',
 'Signal peptidase I P',
 'Rhamnosyl O-methyltransferase',
 '8-oxo-dGTP diphosphatase',
 'putative protein YhaZ',
 'Lysine-specific permease',
 'Uridine kinase',
 'Transcriptional activator protein CopR',
 'Adaptive-response sensory-kinase SasA',
 'HTH-type transcriptional regulator NorG',
 'putative amino-acid metabolite efflux pump',
 'putative protein YqjY',
 'DNA gyrase inhibitor',
 'RNA pyrophosphohydrolase',
 'Tetracycline resistance protein TetO',
 'Transcriptional regulator SlyA',
 'Putative NAD(P)H nitroreductase YodC',
 'Putative monooxygenase YcnE',
 'ATPase RavA',
 'IS4 family transposase MICBce1',
 'Putative adenine deaminase YerA',
 'Col

In [ ]:
rast_fuc = []
with open ('rast.txt') as text:
  for i, line in enumerate(text):
    print(line)
    line = line.strip().split('	')[2]
    if 'hypothetical' not in line:
     rast_fuc.append(line)
print(len(set(set(rast_fuc))))
set(rast_fuc)

fig|2116814.3352.peg.1	14_21_1460	Predicted histidine uptake transporter

fig|2116814.3352.peg.2	14_1573_1911	acetyltransferase, GNAT family

fig|2116814.3352.peg.3	14_1979_2092	acetyltransferase, GNAT family

fig|2116814.3352.peg.4	14_2391_3050	EpiH/GdmH-related protein

fig|2116814.3352.peg.5	14_3289_4437	Butyryl-CoA dehydrogenase (EC 1.3.99.2)

fig|2116814.3352.peg.6	14_5138_4668	FIG01226774: hypothetical protein

fig|2116814.3352.peg.7	14_5208_5915	Transcriptional regulator, ArsR family

fig|2116814.3352.peg.8	14_6348_6803	Glyoxalase family protein

fig|2116814.3352.peg.9	14_7555_8232	Cytoplasmic copper homeostasis protein cutC

fig|2116814.3352.peg.10	14_8256_8807	Signal peptidase I (EC 3.4.21.89)

fig|2116814.3352.peg.11	14_8961_9794	DNA alkylation repair enzyme

fig|2116814.3352.peg.12	14_10124_10237	hypothetical protein

fig|2116814.3352.peg.13	14_10239_10991	Transcriptional regulator, IclR family

fig|2116814.3352.peg.14	14_11006_11719	Allophanate hydrolase 2 subunit 1 (EC 3.5

{'6-aminohexanoate-dimer hydrolase (EC 3.5.1.46)',
 'ATPase, AAA family',
 'Adenine deaminase (EC 3.5.4.2)',
 'Allophanate hydrolase 2 subunit 1 (EC 3.5.1.54)',
 'Allophanate hydrolase 2 subunit 2 (EC 3.5.1.54)',
 'Aminoglycoside phosphotransferase',
 'Butyryl-CoA dehydrogenase (EC 1.3.99.2)',
 'CAAX amino terminal protease family family',
 'Catalase (EC 1.11.1.6)',
 'Cephalosporin hydroxylase',
 'Cytoplasmic copper homeostasis protein cutC',
 'D-alanyl-D-alanine carboxypeptidase (EC 3.4.16.4)',
 'DNA alkylation repair enzyme',
 'EMG2 protein',
 'EpiH/GdmH-related protein',
 'FIG001614: Membrane protein',
 'FIG015373: Membrane protein',
 'Glyoxalase family protein',
 'Lactam utilization protein LamB',
 'Lmo0066 homolog within ESAT-6 gene cluster, similarity to ADP-ribosylating toxins',
 'Lysine-specific permease',
 'Methyltransferase (EC 2.1.1.-)',
 'Microbial collagenase (EC 3.4.24.3)',
 'MutT/nudix family protein',
 'NAD-dependent protein deacetylase of SIR2 family',
 'Nudix dNTPase 

In [ ]:
all_coordinates.shape

(60, 2)

Статистика по RAST (далее в анализ не входили hypothetical proteins): точные функции установлены для 46 генов
Статистика по PROKKA (далее в анализ не входили hypothetical proteins):точные функции установлены для 33 генов
Статистика по аннотации: проаннотировано 60 участков 

Сравнение между prokka и аннотацией: 25 из 33 генов из prokka совпали по своей функции с аннотацией. То есть 8 функций генов нашлись в prokka (соответственно в моей аннотации 60-25 = 35)

Сравнение между rast и аннотацией: совпадения по точным функциям - 7 из 46 (по rast). На самом деле данный анализ проводить сложнее, так как rast не выдавал gene name primary и тогда приходилось смотреть и сравнивать сначала с prokka (вдруг там есть совпадение и оттуда можно сразу получить gene name primary, а это упрощает поиск). Но оказалось, что и RAST с PROKKA между собой не сильно похожи (либо запись функций в rast настолько "завуалированы", что нужно вдаваться в биологию). Соответсвенно,  39 функций нашел rast в отличие от моей аннотации и 53 функции есть в моей аннотации, но нет в rast
 